In [19]:
# Regular imports
import sys
import os

import numpy as np
import scipy.special as sps
import scipy.constants as spc
import scipy.interpolate as spinter
import scipy.optimize as spopt
import scipy.fft as ft
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap, LogNorm
# from matplotlib import rc, rcParams
import time
from datetime import datetime
from PIL import Image

plt.style.use('./xri_plot.mplstyle')
plt.close('all')

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

In [20]:
# import custom modules -> re-run if changed

import images
import instrument
import process
import analysis

In [21]:
# generate PSF of initialised setup

pix_scale = np.array([500, 500])

def generate_psf(energy, counts, img_scale, inst, samples = pix_scale):
    
    point_source = images.point_source(counts, alpha = 0, beta = 0, energy = energy)
    psf_data = process.interferometer_data(inst, point_source)
    psf, uv, obj = analysis.image_recon_smooth(psf_data, inst, img_scale, samples = samples, error=0.05, recon_type = 'IFFT')
    return psf, uv, obj



In [22]:
counts = np.logspace(1, 8, 8, dtype=int)
baselines = np.linspace(0.05, 1.0, 10)
energy = 6.4
img_scale = 500e-6 # width of image in arcsec
time_step = 1





psf_list = []
uv_list = []
for c in counts:
    img = images.point_source(c, alpha = 0, beta = 0, energy = energy)
    inst = instrument.interferometer(time_step, roller = instrument.interferometer.smooth_roller, roll_init = 0, roll_speed = np.pi / (np.max(img.toa)* time_step))
    for i in baselines:
        inst.add_willingale_baseline(i)
    simulated_psf, uv, obj = generate_psf(energy, c, img_scale, inst, samples = pix_scale)
    psf_list.append(simulated_psf)
    uv_list.append(uv)
    print(f'Generated PSF for {c} counts')

# plot PSFs




Generated PSF for 10 counts
Generated PSF for 100 counts
Generated PSF for 1000 counts
Generated PSF for 10000 counts
Generated PSF for 100000 counts
Generated PSF for 1000000 counts
Generated PSF for 10000000 counts
Generated PSF for 100000000 counts


In [23]:
def evaluate_relative_astrometry(psf_list, aperture_radius=50):
    """
    Evaluate the relative astrometry for each PSF in psf_list.
    Uses centroiding within a fixed circular aperture around the image center.

    Parameters:
        psf_list (list): List of PSF arrays (each shape: [N, N]).
        aperture_radius (int): Radius of the circular aperture (pixels).

    Returns:
        centroids (list): List of (y, x) centroid coordinates for each PSF.
        errors (list): List of (σy, σx) centroid uncertainties for each PSF.
    """
    centroids = []
    errors = []
    for psf in psf_list:
        img = psf[0]  # PSF array
        N = img.shape[0]
        y0, x0 = N // 2, N // 2

        # Create aperture mask
        Y, X = np.ogrid[:N, :N]
        mask = (X - x0)**2 + (Y - y0)**2 <= aperture_radius**2

        # Apply mask
        img_ap = img * mask

        # Centroid calculation
        total = img_ap.sum()
        y_centroid = (img_ap * Y).sum() / total
        x_centroid = (img_ap * X).sum() / total

        # Error estimation (variance of centroid, assuming Poisson statistics)
        # σ² = Σ[(pixel - centroid)² * value] / total²
        y_var = ((img_ap * (Y - y_centroid)**2).sum()) / total**2
        x_var = ((img_ap * (X - x_centroid)**2).sum()) / total**2

        centroids.append((y_centroid, x_centroid))
        errors.append((np.sqrt(y_var), np.sqrt(x_var)))
    return centroids, errors


centroids, errors = evaluate_relative_astrometry(psf_list, aperture_radius=50)
for i, (y, x) in enumerate(centroids):
    print(f'Counts: {counts[i]:.0f}, Centroid: (y={y:.2f}, x={x:.2f}), Error: (σy={errors[i][0]:.2f}, σx={errors[i][1]:.2f})')


Counts: 10, Centroid: (y=253.09, x=263.32), Error: (σy=828.88, σx=1492.53)
Counts: 100, Centroid: (y=253.02, x=251.05), Error: (σy=375.24, σx=465.69)
Counts: 1000, Centroid: (y=248.76, x=250.16), Error: (σy=117.61, σx=121.38)
Counts: 10000, Centroid: (y=250.05, x=249.56), Error: (σy=37.23, σx=37.77)
Counts: 100000, Centroid: (y=249.74, x=250.11), Error: (σy=11.83, σx=11.81)
Counts: 1000000, Centroid: (y=249.97, x=250.01), Error: (σy=3.76, σx=3.75)
Counts: 10000000, Centroid: (y=250.01, x=250.02), Error: (σy=1.19, σx=1.19)
Counts: 100000000, Centroid: (y=250.00, x=250.00), Error: (σy=0.38, σx=0.38)


In [24]:
import os
plt.close('all')
# Create output directory if it doesn't exist
output_dir = 'Figures/astrometry'
os.makedirs(output_dir, exist_ok=True)

aperture_radius = 50  # same as used in evaluate_relative_astrometry
powers = np.log10(counts)

# Plot all PSFs in the same style as ps_test.ipynb
for idx, (psf, _) in enumerate(psf_list):
    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(psf/psf.max(), cmap='inferno', origin='lower')
    ax.set_title(f'Point source with $10^{powers[idx]:.0f}$ counts')
    ax.set_xlabel('x [µas]')
    ax.set_ylabel('y [µas]')
    plt.colorbar(im, ax=ax, label='Intensity (arbitrary unit)', pad=0.02, fraction=0.046)

    # Draw aperture circle
    N = psf.shape[0]
    y0, x0 = N // 2, N // 2
    circ = plt.Circle((x0, y0), aperture_radius, color='cyan', fill=False, lw=2, linestyle='--')
    ax.add_patch(circ)
    # Add text annotation for aperture
    ax.text(x0, y0 + aperture_radius + 20, '50px aperture', color='cyan', ha='center', va='bottom', fontsize=12)

    fig.tight_layout()
    fig.savefig(f'{output_dir}/psf_{counts[idx]:.0f}_counts.png', dpi=200)
    plt.close(fig)

In [25]:
from scipy.optimize import curve_fit
# Prepare data
errors_arr = np.array(errors)
mean_errors = errors_arr.mean(axis=1)
diff_errors = np.abs(errors_arr[:, 0] - errors_arr[:, 1])

# Power-law fit function
def powerlaw(x, a, b):
    return a + x**b

def linear(x, m, c):
    return m * x + c

popt, pcov = curve_fit(linear, np.log10(counts), np.log10(mean_errors), sigma=diff_errors, absolute_sigma=True)
perr = np.sqrt(np.diag(pcov))
print(f'Fitted power-law parameters: slope = {popt[0]:.2f} ± {perr[0]:.2f}, intercept = {popt[1]:.2f} ± {perr[1]:.2f}')
# Plot
fig2, ax2 = plt.subplots(figsize=(7, 5))
ax2.errorbar(counts, mean_errors, yerr=diff_errors, fmt='s', color='tab:blue', label='Mean centroid error')
ax2.plot(counts, 10**linear(np.log10(counts), *popt), 'r-', label=f'Fitted slope $= {popt[0]:.2f} \pm {perr[0]:.2f}$')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Counts')
ax2.set_ylabel('Mean centroid error (pixels)')
ax2.legend(fontsize='small')
fig2.tight_layout()
fig2.savefig(f'{output_dir}/centroid_error_vs_counts.png', dpi=200)
plt.close(fig2)

<>:20: SyntaxWarning: invalid escape sequence '\p'
<>:20: SyntaxWarning: invalid escape sequence '\p'
/var/folders/fn/72t1j4cn2b7gn_2dhs83dn_c0000gn/T/ipykernel_64527/3410113784.py:20: SyntaxWarning: invalid escape sequence '\p'
  ax2.plot(counts, 10**linear(np.log10(counts), *popt), 'r-', label=f'Fitted slope $= {popt[0]:.2f} \pm {perr[0]:.2f}$')


Fitted power-law parameters: slope = -0.50 ± 0.00, intercept = 3.57 ± 0.01
